# 3 Ewaluacja, regularyzacja, optymalizacja

## 3.1 Ewaluacja

Plik mushrooms.tsv zawiera dane dotyczące różnych gatunków grzybów pochodzące ze strony
https://archive.ics.uci.edu/ml/datasets/mushroom.
W pierwszej kolumnie znajduje się informacja, czy grzyb jest jadalny (e – edible) czy trujący (p – poisonous).
Pozostałe kolumny zawierają wartości poszczególnych cech (opis pod powyższym linkiem).

Za pomocą dowolnego algorytmu klasyfikacji, np. regresji logistycznej z pakietu scikit-learn,
stwórz model, który przewidzi na podstawie kilku wybranych cech (nie wszystkich), czy grzyb jest jadalny, czy trujący:

Przed przystąpieniem do budowy modelu podziel dane na zbiór uczący i zbiór testowy.
Po wytrenowaniu modelu na zbiorze uczącym dokonaj jego ewaluacji na zbiorze testowym - oblicz
dokładność (accuracy), precyzję (precision), pokrycie (recall) i F-score.

## Import Libraries and Dataset

In [113]:
# IMPORT LIBRARIES
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

# IMPORT DATASET
full_df = pd.read_csv('mushrooms.tsv',sep = '\t', header=None)

full_df = full_df.rename(columns={
            0 : 'edibility', #  edible=e, poisonous=p
            1 : 'cap-shape', #  bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s
            2 : 'cap-surface', # fibrous=f, grooves=g, scaly=y, smooth=s
            3 : 'cap-color', # brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y
            4 : 'bruises?', # bruises=t,no=f
            5 : 'odor', # almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s
            6 : 'gill-attachment', # attached=a, descending=d, free=f, notched=n
            7 : 'gill-spacing', # close=c, crowded=w, distant=d
            8 : 'gill-size', # broad=b, narrow=n
            9 : 'gill-color', # black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u, red=e, white=w, yellow=y
            10 : 'stalk-shape', # enlarging=e, tapering=t
            11 : 'stalk-root', # bulbous=b, club=c, cup=u, equal=e, rhizomorphs=z, rooted=r, missing=?
            12 : 'stalk-surface-above-ring', # fibrous=f, scaly=y, silky=k, smooth=s
            13 : 'stalk-surface-below-ring', # fibrous=f, scaly=y, silky=k, smooth=s
            14 : 'stalk-color-above-ring', # brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
            15 : 'stalk-color-below-ring', # brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
            16 : 'veil-type', # partial=p, universal=u
            17 : 'veil-color', # brown=n, orange=o, white=w, yellow=y
            18 : 'ring-number', # none=n, one=o, two=t
            19 : 'ring-type', # cobwebby=c, evanescent=e, flaring=f, large=l, none=n, pendant=p, sheathing=s, zone=z
            20 : 'spore-print-color', # black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y
            21 : 'population', # abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y
            22 : 'habitat' # grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w, woods=d
            })
df = full_df[['edibility', 'cap-surface', 'cap-color', 'gill-size']].copy()
df.sample(5)

,edibility,cap-surface,cap-color,gill-size
867,e,s,w,b
2771,e,f,e,b
6392,e,s,n,b
4973,e,f,n,n
1801,p,y,w,n


## Check for NaN under an entire DataFrame

In [114]:
df.isnull().values.any()

False

## List Unique Values

In [115]:
df.edibility.unique()

array(['e', 'p'], dtype=object)

## Feature Engineering (One-Hot Encoding)

In [116]:
df = pd.get_dummies(df, drop_first=True)
df.sample(5)

,edibility_p,cap-surface_g,cap-surface_s,cap-surface_y,cap-color_c,cap-color_e,cap-color_g,cap-color_n,cap-color_p,cap-color_r,cap-color_u,cap-color_w,cap-color_y,gill-size_n
1940,1,0,1,0,0,0,0,1,0,0,0,0,0,1
1019,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1693,0,0,0,0,0,0,0,1,0,0,0,0,0,0
6356,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3143,0,0,0,0,0,0,0,1,0,0,0,0,0,0


## Splitting Data into Training and Test Sets (Digits Dataset)

In [117]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('edibility_p', axis=1), df['edibility_p'])

## Train the model using the training data

In [118]:
LogReg = LogisticRegression(solver='lbfgs')
LogReg.fit(X_train, y_train)

LogisticRegression()

## Scoring the model

In [119]:
LogReg.score(X_test, y_test)

0.8055096418732782

### Understanding the score

In [120]:
pred = (LogReg.predict(X_test) > .5).astype(int)
np.sum(pred == y_test) / len(y_test)

0.8055096418732782

## Classification metrics

In [121]:
prediction = LogReg.predict(X_test)

accuracy = metrics.accuracy_score(y_test, prediction)
precision = metrics.precision_score(y_test, prediction)
recall = metrics.recall_score(y_test, prediction)
fscore = metrics.f1_score(y_test, prediction)


print('Accuracy: %f' % accuracy) # dokładność
print('Precision: %f' % precision) # precyzja
print('Recall: %f' % recall) # pokrycie
print('F-score: %f' % fscore) # F-score

Accuracy: 0.805510
Precision: 0.816448
Recall: 0.774887
F-score: 0.795125
